In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
test_rule = pd.read_csv('submit/submit_rule.csv')

In [7]:
test_rule['ymd'] = test_rule.time_stamp.apply(lambda x: x[:-3])
test_rule['hour'] = test_rule.time_stamp.apply(lambda x: int(x[-2:]))
test_rule['month'] = test_rule['ymd'].apply(lambda x: int(x[5:7]))
test_rule = test_rule.rename(columns={'num_of_people':'num'})

In [5]:
test_model = pd.read_csv('submit/submit_gbm_normal_lgb.csv')

In [8]:
test_model['ymd'] = test_model.time_stamp.apply(lambda x: x[:-3])
test_model['hour'] = test_model.time_stamp.apply(lambda x: int(x[-2:]))
test_model['month'] = test_model['ymd'].apply(lambda x: int(x[5:7]))
test_model = test_model.rename(columns={'num_of_people':'num'})

In [9]:
test_blend = test_rule.copy()

In [10]:
test_blend['num2'] = test_model['num']

In [11]:
sum(test_blend['num'] < test_blend['num2'])/len(test_blend)

0.41572987943955686

In [12]:
test_blend['xiaoyu'] = (test_blend['num'] < test_blend['num2']).astype(int)

In [13]:
test_blend['fin_num'] = 0.0

In [14]:
# 规则预测偏小的结果保留
test_blend.loc[test_blend.xiaoyu==1, 'fin_num'] = test_blend.loc[test_blend.xiaoyu==1, 'num'] 

In [15]:
# 规则预测偏大的结果与模型预测结果融合
test_blend.loc[test_blend.xiaoyu==0, 'fin_num'] = \
test_blend.loc[test_blend.xiaoyu==0, 'num'] * 0.75 + 0.25 * test_blend.loc[test_blend.xiaoyu==0, 'num2']

In [16]:
test_blend = test_blend[['loc_id','fin_num','ymd','hour','month', 'time_stamp']].rename(columns={'fin_num':'num'})

In [93]:
# 乘以修正系数
test_blend['num'] = test_blend['num'] * 0.960209838948603

In [133]:
# 修正14地点的一些异常预测情况
test_blend.loc[(test_blend.loc_id==14)&(test_blend.hour<=6)&(test_blend.month==12), 'num'] = \
test_blend.loc[(test_blend.loc_id==14)&(test_blend.hour<=6)&(test_blend.month==12)]['num'].apply(lambda x: x*0.45)

In [144]:
test_blend = test_blend.rename(columns={'num':'num_of_people'})

In [146]:
test_blend[['loc_id', 'time_stamp', 'num_of_people']].to_csv('submit/submit_blend_1600.csv', index=None)